In [ ]:
# ArrayList and Stack code to use

class Stack():
    def __init__(self):
        self.inList = None
        self.size = 0
        
    def push(self, v):
        self.size += 1
        self.inList = (v,self.inList)
        
    def pop(self):
        if self.size == 0: assert(0), "Popping from an empty Stack!"
        self.size -= 1
        (v,ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v,ls) = ls
            s += str(v)
            if ls!=None: s += ", "
        return s+"]"
        
        
class ArrayList:
    def __init__(self):
        self.inArray = [0 for i in range(10)]
        self.count = 0
        
    def get(self, i):
        return self.inArray[i]

    def set(self, i, e):
        self.inArray[i] = e

    def length(self):
        return self.count

    def append(self, e):
        self.inArray[self.count] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        for j in range(self.count,i,-1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        self.count -= 1
        val = self.inArray[i]
        for j in range(i,self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        newArray = [0 for i in range(2*len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        return self.inArray[:self.count]

    def __str__(self):
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i])+", "
        return s+str(self.inArray[self.count-1])+"]" 


class ArrayListWithUndo(ArrayList):
    def __init__(self):
        # already implemented
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        # already implemented
        self.undos.push(("set", i, self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        self.undos.push(("rem", self.length(), None))
        super().append(v)
        
    def insert(self, i, v):
        self.undos.push(("rem", i, None))
        super().insert(i, v)
    
    def remove(self, i):
        v = self.get(i)
        self.undos.push(("ins", i, v))
        super().remove(i)
    
    def undo(self):
        if self.undos.size == 0:
            return
        undoOperation = self.undos.pop()
        op = undoOperation[0]
        i = undoOperation[1]
        v = undoOperation[2]
        if op == "set":
            self.inArray[i] = v
        elif op == "ins":
            super().insert(i, v)
        elif op == "rem":
            super().remove(i)

    def toArray(self):
        return [super().get(i) for i in range(self.count)]
            
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.undos)
        

class NetworkWithUndo:
    def __init__(self, N):
        # already implemented
        self.inArray = ArrayListWithUndo()
        for _ in range(N): self.inArray.append(-1)
        self.undos = Stack()
        self.undos.push(N)
        
    def getSize(self):
        # already implemented
        return self.inArray.length()
        
    def add(self): 
        self.inArray.append(-1)
        self.undos.push(1)
    
    def root(self, i):
        self.visited = Stack()
        root = self._findRoot(i)
        if self.visited.size == 0:
            self.undos.push(0)
            return root
        self.visited.pop() # popping node already pointing to root
        self.undos.push(self.visited.size)
        while self.visited.size > 0:
            self.inArray.set(self.visited.pop(), root)
        return root
    
    def _findRoot(self, i):
        node = self.inArray.get(i)
        if node < 0:
            return i
        self.visited.push(i)
        return self._findRoot(node)
    
    def merge(self, i, j):
        iValue = self.inArray.get(i)
        jValue = self.inArray.get(j)
        if iValue > -1 or jValue > -1:
            assert(0)
        if i == j:
            self.undos.push(0)
            return
        newSize = iValue + jValue
        if iValue >= jValue:
            self.inArray.set(i, j)
            self.inArray.set(j, newSize)
        else:
            self.inArray.set(j, i)
            self.inArray.set(i, newSize)
        self.undos.push(2)
    
    def undo(self):
        if self.undos.size == 0:
            return
        numOfUndos = self.undos.pop()
        for _ in range(numOfUndos):
            self.inArray.undo()
    
    def toArray(self):
        # already implemented
        return self.inArray.toArray()
            
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.undos)
    

class Gadget:
    def __init__(self):
        # already implemented
        self.inNetwork = NetworkWithUndo(0)
        self.subsize = 0
        self.nameMap = {}
        self.undos = Stack()
        self.helper = {}
        
    def getSize(self):
        # already implemented
        return self.inNetwork.getSize()
        
    def isIn(self, name):
        # already implemented
        return name in self.nameMap
        
    def add(self, name):
        if self.isIn(name):
            return
        self.nameMap[name] = self.inNetwork.getSize()
        self.inNetwork.add()
        self.subsize += 1
        self._incrementHelper()
        self.helper[name] = 1
        self.undos.push(("rem", 1, name))
    
    def connect(self, name1, name2):
        root1 = self.inNetwork.root(self.nameMap[name1])
        root2 = self.inNetwork.root(self.nameMap[name2])
        if root1 == root2:
            self.undos.push(("oth", 2, None))
            return True
        self.inNetwork.merge(root1, root2)
        self.subsize -= 1
        self._incrementHelper()
        self.undos.push(("brk", 3, None))
        return False

    def clean(self, name):
        undoCount = self.helper[name]
        self.undo(undoCount)

    def _incrementHelper(self):
        for name in self.helper:
            self.helper[name] += 1

    def _decreaseHelper(self):
        for name in self.helper:
            self.helper[name] -= 1
            if self.helper[name] == 0:
                self._removeNameFromHelper(name)

    def _removeNameFromHelper(self, nameToRemove):
        newMap = {}
        for name in self.helper:
            if name != nameToRemove:
                newMap[name] = self.helper[name]
        self.helper = newMap

    def subnets(self):
        subnetsMap = {}
        namesArr = [name for name in self.nameMap]
        for i in range(self.inNetwork.getSize()):
            root = self.inNetwork.root(i)
            if root not in subnetsMap:
                subnetsMap[root] = ArrayList()
            subnetsMap[root].append(namesArr[i])
        self.undos.push(("oth", self.inNetwork.getSize(), None))
        self._incrementHelper()  
        subnetsArr = [None for _ in range(len(subnetsMap))]
        i = 0
        for subnet in subnetsMap:
            nodesArrayList = subnetsMap[subnet]
            nodes = nodesArrayList.inArray[:nodesArrayList.count]
            subnetsArr[i] = nodes
            i += 1
        return subnetsArr
        
    def undo(self, count):
        for _ in range(count):
            if self.undos.size == 0:
                return
            undoOperation = self.undos.pop()
            op = undoOperation[0]
            n = undoOperation[1]
            s = undoOperation[2]
            if op == "rem":
                self._rem(s)
            elif op == "brk":
                self._brk(n)
            elif op == "oth":
                self._oth(n)

    def _rem(self, s):
        self.inNetwork.undo()
        self.subsize -= 1
        self._removeNameFromNameMap(s)
        self._decreaseHelper()

    def _removeNameFromNameMap(self, nameToRemove):
        newMap = {}
        for name in self.nameMap:
            if name != nameToRemove:
                newMap[name] = self.nameMap[name]
        self.nameMap = newMap

    def _brk(self, n):
        self._decreaseHelper()
        for _ in range(n):
            self.inNetwork.undo()
        self.subsize += 1

    def _oth(self, n):
        self._decreaseHelper()
        for _ in range(n):
            self.inNetwork.undo()
        
    def toArray(self):
        # already implemented
        A = self.inNetwork.toArray()
        for s in self.nameMap:
            i = self.nameMap[s]
            A[i] = (s,A[i])
        return A
        
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.nameMap)+"\n-> "+str(self.undos)